In [2]:
import numpy as np
import h5py
import CAS.dmrghandler.src.dmrghandler.hdf5_io as hdf5_io
import CAS.dmrghandler.src.dmrghandler.dmrg_processing as dp
from pathlib import Path
import openpyxl as px
import openpyxl.chart as px_chart
import pandas as pd

In [ ]:
"""
# fcidumps_blocksize5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size5
sbatch ../8eeb7874-4212-49d0-920a-8d7fc7ddb16b/submit_8eeb7874-4212-49d0-920a-8d7fc7ddb16b.sh
# fcidumps_blocksize5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_size5
sbatch ../2bc57fb8-9d5b-4a81-a2d2-f1b6b235cdd8/submit_2bc57fb8-9d5b-4a81-a2d2-f1b6b235cdd8.sh
# fcidumps_blocksize5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_hidden_size5
sbatch ../85e89faf-4541-48f8-92ee-025b54010a08/submit_85e89faf-4541-48f8-92ee-025b54010a08.sh
# fcidumps_blocksize5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_hidden_size5
sbatch ../9ca318c0-348c-4bd9-be8d-0c13d66f35d6/submit_9ca318c0-348c-4bd9-be8d-0c13d66f35d6.sh
# fcidumps_blocksize5/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_hidden_size5
sbatch ../c772afcc-15ab-407b-94f7-55728306578b/submit_c772afcc-15ab-407b-94f7-55728306578b.sh
# fcidumps_blocksize5/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_hidden_size5
sbatch ../df6bde80-8319-47c3-8c67-f0f0386446bb/submit_df6bde80-8319-47c3-8c67-f0f0386446bb.sh
# fcidumps_blocksize5/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_size5
sbatch ../bed5b6b3-d02e-4bb0-8791-698084863f17/submit_bed5b6b3-d02e-4bb0-8791-698084863f17.sh

# fcidumps_blocksize3/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_hidden_size3
sbatch ../175ab035-f0bd-4182-a7cd-48e392d6717b/submit_175ab035-f0bd-4182-a7cd-48e392d6717b.sh


# fcidumps_blocksize3_5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_hidden_size3_5
sbatch ../e5abaaa0-e6b0-4859-bf0e-977fa234d7de/submit_e5abaaa0-e6b0-4859-bf0e-977fa234d7de.sh
# fcidumps_blocksize3_5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_hidden_size3_5
sbatch ../279f5661-549f-447c-8dd2-fba4bbc45e74/submit_279f5661-549f-447c-8dd2-fba4bbc45e74.sh
# fcidumps_blocksize3_5/fcidump.23_pc-_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_size3_5
sbatch ../d00b2cc1-e5c3-4955-9239-00e6cd0e42b8/submit_d00b2cc1-e5c3-4955-9239-00e6cd0e42b8.sh
# fcidumps_blocksize3_5/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_hidden_size3_5
sbatch ../4d6bfae1-47fc-43e9-8e77-090c97d6c38b/submit_4d6bfae1-47fc-43e9-8e77-090c97d6c38b.sh
# fcidumps_blocksize3_5/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_hidden_size3_5
sbatch ../479f0d62-f1e9-45f8-ae1d-268515beef70/submit_479f0d62-f1e9-45f8-ae1d-268515beef70.sh
# fcidumps_blocksize3_5/fcidump.27_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_3_size3_5
sbatch ../dc828757-5382-465f-9f93-97e755b29468/submit_dc828757-5382-465f-9f93-97e755b29468.sh

"""

In [3]:
catalyst_meta_data_path=Path('../catalysis_instances.csv')
excel_name=Path('20240613_var_size55.xlsx')
data_file_path = Path("../dmrg_data/data_storage_size_51")

run_commands = \
"""
# up to 20 log10 hilbert space
#######################
# fcidumps_blocksize_55/fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size551
sbatch ../f5754056-a677-4e36-90bd-d95379839df3/submit_f5754056-a677-4e36-90bd-d95379839df3.sh
# fcidumps_blocksize_55/fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size552
sbatch ../b924927e-0e93-478d-865f-988e21f7454c/submit_b924927e-0e93-478d-865f-988e21f7454c.sh
# fcidumps_blocksize_55/fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size553
sbatch ../e33b33e7-268b-4887-8f71-4c84c2beaad8/submit_e33b33e7-268b-4887-8f71-4c84c2beaad8.sh
# fcidumps_blocksize_55/fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size554
sbatch ../72be5500-1cea-4942-b05a-654ccf60223b/submit_72be5500-1cea-4942-b05a-654ccf60223b.sh

"""

In [5]:
slurm_emails_folder = Path('../slurm_emails/slurm_emails_size5')
fci_dump_folder  = Path("../fcidumps_directory/fcidumps_blocksize_51")
excel_storage_path = Path("../excel_data_storage")
csv_storage_path = excel_storage_path/Path("20240613performance_metrics_blocksize55")

In [6]:
catalyst_meta_data = pd.read_csv(catalyst_meta_data_path)

In [7]:
cc_wall_time_dict = {}
# Get all eml files in the folder
slurm_email_files = list(slurm_emails_folder.glob('*.eml'))
for email_file in slurm_email_files:
    with open(email_file, 'r') as f:
        use_next_line_subject = False
        obtained_run_time = False
        for line in f:
            if use_next_line_subject:
                line = line.strip()
                run_time = line.split(', ')[0].split(' ')[-1]
                # print(run_time)
                obtained_run_time = True
                use_next_line_subject = False
            if line.startswith('Subject: '):
                use_next_line_subject = True
                # print(line)
            elif line.startswith('   Command=../'):
                line = line.strip()
                uuid = line.split('/')[1]
                # print(uuid)
                assert obtained_run_time, 'Run time not yet obtained'
                cc_wall_time_dict[uuid] = run_time
                
print(cc_wall_time_dict)

{'e15ea3e4-d312-4df3-afd6-5925a1012f4f': '00:03:07', '475d062c-b6f4-461a-aaac-1fb709544a80': '00:02:22', 'b924927e-0e93-478d-865f-988e21f7454c': '00:09:07', 'aa795cfe-1b10-4982-aa83-6241c04e0bc5': '00:02:49', '246fcbb7-d062-4781-8411-9526f2693e3d': '00:03:17', '4930391a-809d-4dbe-bd8f-c2be3ce68214': '00:08:00', '72be5500-1cea-4942-b05a-654ccf60223b': '00:10:31', '1b0b89c4-656c-4d01-9781-551c73dbfa03': '00:02:53', '6cbe0ecf-bad6-4737-9a81-4811dea0352b': '00:02:53', 'e3556631-b2ae-4d65-9a5f-096b02112366': '00:02:19', 'f5754056-a677-4e36-90bd-d95379839df3': '00:02:49', 'e33b33e7-268b-4887-8f71-4c84c2beaad8': '00:03:22', '4556cfff-fcc7-4d01-a051-02a4c66e7349': '00:02:22', '8dfda386-e4ae-4063-855a-456831cc5369': '00:07:17'}


In [8]:
#Load all fcidump file names
fcidump_files = [filename.name for filename in list(fci_dump_folder.glob('fcidump.*'))]
# fcidump_files

In [9]:
import io
buf = io.StringIO(run_commands)
orig_data_dict_list = []
for line in buf.readlines():
    # print(line)
    if line.startswith('### '):
        continue
    if line.startswith('# fcidumps_blocksize_55/'):
        fcidump_name_temp = line.split('fcidumps_blocksize_55/')[1].strip()
        dict_entry = {}
        # print(fcidump_name_temp)
        # if fcidump_name.endswith('}'):
        #     fcidump_test_string = fcidump_name.split('{')[0]
        # else:
        #     basis_end = fcidump_name.split('_')[-1]
        #     fcidump_test_string = fcidump_name.split('_'+basis_end)[0]+'_'
        fcidump_test_string = fcidump_name_temp.split('_')[0]+'_'
        # print(fcidump_test_string)
        for fcidump_name in fcidump_files:
            if fcidump_name.startswith(fcidump_test_string):
                break
        print(fcidump_name_temp)
        dict_entry['fcidump'] = fcidump_name_temp
        dict_entry['fcidump_test_string'] = fcidump_test_string
    if line.startswith('sbatch ../'):
        calc_uuid = line.split('sbatch ../')[1].split('/')[0]
        # print(calc_uuid)
        dict_entry['Calc UUID'] = calc_uuid
        dict_entry['CC Wall Time'] = cc_wall_time_dict[calc_uuid]
        dict_entry['Attempt Result'] = ""
        orig_data_dict_list.append(dict_entry)

print(orig_data_dict_list)  

fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size551
fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size552
fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size553
fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size554
[{'fcidump': "fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size551", 'fcidump_test_string': 'fcidump.26_', 'Calc UUID': 'f5754056-a677-4e36-90bd-d95379839df3', 'CC Wall Time': '00:02:49', 'Attempt Result': ''}, {'fcidump': "fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_size552", 'fcidump_test_string': 'fcidump.26_', 'Calc UUID': 'b924927e-0e93-478d-865f-988e21f7454c', 'CC Wall Time': '00:09:07', 'Attempt Result': ''}, {'fcidump': "fcidump.26_pc_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_

In [12]:
data_dict_list = []
for data_dict in orig_data_dict_list:
    new_data_dict = {}
    fcidump_test_string = data_dict['fcidump_test_string']
    #Get metadata row
    data_row = catalyst_meta_data[catalyst_meta_data['fcidump'].str.contains(fcidump_test_string)]
    new_data_dict['instance ID'] = data_row['instance ID'].values[0]
    new_data_dict['molecule'] = data_row['molecule'].values[0]
    new_data_dict['charge'] = data_row['charge'].values[0]
    new_data_dict['multiplicity'] = data_row['multiplicity'].values[0]
    new_data_dict['fcidump'] = data_dict['fcidump']
    new_data_dict['log10_hilbert_space_size'] = data_row['log10_hilbert_space_size'].values[0]
    new_data_dict['Attempt Result'] = data_dict['Attempt Result']
    new_data_dict['Calc UUID'] = data_dict['Calc UUID']
    new_data_dict['CC Wall Time'] = data_dict['CC Wall Time']
    if 'Calc UUID Small BD' in data_dict:
        new_data_dict['Calc UUID Small BD'] = data_dict['Calc UUID Small BD']
        new_data_dict['CC Wall Time Small BD'] = data_dict['CC Wall Time Small BD']
    data_dict_list.append(new_data_dict)


In [13]:
wb = px.Workbook()

dp.setup_workbook(data_file_path=data_file_path,data_dict_list=data_dict_list,workbook=wb,csv_storage_path=csv_storage_path)

wb.save(excel_storage_path/excel_name)

Last loop included = 1
Processed results available
Checking that processed results match raw results.
Last loop included = 63
Processed results available
Checking that processed results match raw results.
Last loop included = 6
Processed results available
Checking that processed results match raw results.
Last loop included = 63
Processed results available
Checking that processed results match raw results.


In [416]:
"""
# fcidumps_catalysts_larger_system/fcidump.52_6acme_{'default'_'6-31++G(d,p)'}_tbt_1_hidden
sbatch ../64f1d666-d92f-41c0-b5dc-aa8c1bf9820b/submit_64f1d666-d92f-41c0-b5dc-aa8c1bf9820b.sh
# fcidumps_catalysts_larger_system/fcidump.52_6acme_{'default'_'6-31++G(d,p)'}_tbt_3_hidden
sbatch ../274010c1-5381-4eb2-a269-6a5d071d1a75/submit_274010c1-5381-4eb2-a269-6a5d071d1a75.sh
# fcidumps_catalysts_larger_system/fcidump.41_1_ts_{'default'_'cc-pVTZ'_'Ru'_'LANL2TZ'}_tbt_1_hidden
sbatch ../35616e05-1242-4bb6-beab-e86f140e5c99/submit_35616e05-1242-4bb6-beab-e86f140e5c99.sh
"""

"\n# fcidumps_catalysts_larger_system/fcidump.52_6acme_{'default'_'6-31++G(d,p)'}_tbt_1_hidden\nsbatch ../64f1d666-d92f-41c0-b5dc-aa8c1bf9820b/submit_64f1d666-d92f-41c0-b5dc-aa8c1bf9820b.sh\n# fcidumps_catalysts_larger_system/fcidump.52_6acme_{'default'_'6-31++G(d,p)'}_tbt_3_hidden\nsbatch ../274010c1-5381-4eb2-a269-6a5d071d1a75/submit_274010c1-5381-4eb2-a269-6a5d071d1a75.sh\n# fcidumps_catalysts_larger_system/fcidump.41_1_ts_{'default'_'cc-pVTZ'_'Ru'_'LANL2TZ'}_tbt_1_hidden\nsbatch ../35616e05-1242-4bb6-beab-e86f140e5c99/submit_35616e05-1242-4bb6-beab-e86f140e5c99.sh\n"